In [1]:
import os
import json
from glob import glob
import pandas as pd
os.chdir("..")

In [2]:
path = "/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/calculate_area individual files"
json_list = glob('/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/*.json')
json_list

['/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/get_weather_info.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/power.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/get_capital.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/calculate_trip_cost.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/count_letter.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/calculate_bmi.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/get_timezone.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/calculate_area.json']

In [3]:
queries = []
answers = []
for file_path in json_list[0:]:
    with open(file_path, 'r') as file:
        text = json.load(file)
        text = text["queries"]
        for item in text:
            queries.append(item["query"])
            answers.append(json.dumps(item["answer"]))
        

In [4]:
import pandas as pd

# check schema 
df = pd.DataFrame({"queries": queries, "answers": answers})
df["functions"] = df["answers"].apply(lambda x: json.loads(x).get("function", None))


print(f"there are {len(df)} rows at the beginning")

there are 5204 rows at the beginning


In [5]:
#filtered non function
df = df[~df["functions"].isna()]

function_names = df["functions"].unique()

sample_list = []
for func_name in function_names:
    sample_df  = df[df["functions"]== func_name].sample(n=210, random_state=1)
    sample_list.append(sample_df)
df = pd.concat(sample_list)

print(f"there are {len(df)} after filtering out non function and sampling")


there are 1680 after filtering out non function and sampling


In [6]:
# filter arguments
df["arguments"] = df["answers"].apply(lambda x: json.loads(x).get("arguments", None))
df = df[~df["arguments"].isna()]
df["arguments"] = df["arguments"].apply(json.dumps)

print(f"there are {len(df)} has arguments keys")


there are 1680 has arguments keys


In [ ]:
from src.chi_functions import *
from src.edward_functions import *
from src.MT_functions import * 
from src.kevin_function import * 
import swifter

def call_function(info: pd.DataFrame):
    try:
        func_to_call = eval(info["functions"])
        
        arguments = json.loads(info["arguments"])
        return func_to_call(**arguments)
    except:
        return None
df["callable"] = df.swifter.apply(lambda x: call_function(x), axis=1)
df = df[~df["callable"].isna()]

len(df)



Sentiment Analysis Result: {"result": {"polarity": 1.0, "subjectivity": 0.75}}
Difference of Squares Result: {"result": 84}
Generated Password: {"result": "h{eC3.FAx1[6q'GD"}
Interest result:  {"result": 400.0}


/Users/cs/Library/Caches/pypoetry/virtualenvs/function-calling-project-_d8EX7TE-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# save DataFrame
df.reset_index(drop=True, inplace=True)
df.to_csv("df_checkpoint_1.csv", index=None)

print(df["functions"].unique())
df[df["functions"] == "calculate_area"]


['get_weather_info' 'power' 'get_capital' 'calculate_trip_cost'
 'count_letter' 'calculate_bmi' 'get_timezone' 'calculate_area']


,queries,answers,functions,arguments,callable
1470,What is the area of a rectangle with length 2....,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""2.14"", ""width"": ""8.93"", ""round_res...","{""area"": ""19""}"
1471,What is the area of a square with length 7.654...,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""7.654"", ""width"": ""7.654"", ""round_r...","{""area"": ""59""}"
1472,What is the area of a rectangle with length 4....,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""4.567"", ""width"": ""7.890"", ""round_r...","{""area"": ""36""}"
1473,What is the area of a square with length 6.8 a...,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""6.8"", ""width"": ""1.5"", ""round_resul...","{""area"": ""10""}"
1474,What is the area of a square with length 8 and...,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""8"", ""width"": ""4.9"", ""round_result""...","{""area"": ""39""}"
...,...,...,...,...,...
1675,What is the area of a rectangle with length 7....,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""7.899"", ""width"": ""8"", ""round_resul...","{""area"": ""63""}"
1676,What is the area of a square with length 2.615...,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""2.615"", ""width"": ""8"", ""round_resul...","{""area"": ""21""}"
1677,Calculate the area of a square with length 8.9...,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""8.901"", ""width"": ""None"", ""round_re...","{""error"": ""Width must be a number.""}"
1678,What is the area of a rectangle with length 5....,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""5.005"", ""width"": ""8"", ""round_resul...","{""area"": ""40""}"


In [6]:
import boto3
from langchain_aws import ChatBedrock
from botocore.config import Config
import time
from tqdm import tqdm
import random

def generate_response(df, llm):
    query_output = df["queries"]
    function_output = df["callable"]
    messages = [
    ("system", f"You are a helpful bot and you concisely answer user's query. Here is the information that can help you answer the question: {function_output}. Do not generate a response more than 25 words"),
    ("user", f"{query_output}")]
    response = llm.invoke(messages)

    return response.content

retry_config = Config(
    retries = {
        'max_attempts': 20,  # Customize the number of retry attempts
        'mode': 'adaptive'   # Or 'adaptive' for dynamically adjusting retries
    }
)

# Initialize the Bedrock client with retry configuration
bedrock_client = boto3.client('bedrock-runtime', config=retry_config)

# Use the ChatBedrock model in LangChain with the client that has retries
llm = ChatBedrock(
    client=bedrock_client,
    model_id="meta.llama3-1-8b-instruct-v1:0",  # Or "meta.llama3-1-70b-instruct-v1:0"
    temperature=0.2,
    max_tokens=None)



responses = []
numbers = []
# os.remove("responses.csv"
for item in tqdm(df.iterrows()):
    try:
        response = generate_response(item[1], llm)
        responses.append(response)
        numbers.append(item[0])
        time.sleep(random.random())
        if item[0] % 5 == 0 :
            temp_df = pd.DataFrame({"index": numbers, "responses": responses})
            temp_df.to_csv('responses.csv', mode='a', index=False, header=False)
            numbers = []
            responses = []
            

    except Exception as e:
        print(e)
        time.sleep(random.random()*70)
        response = generate_response(item[1], llm)
        responses.append(response)
        numbers.append(item[0])
        if item[0] % 5 ==0 :
            temp_df = pd.DataFrame({"index": numbers, "responses": responses})
            temp_df.to_csv('response.csv', mode='a', index=False, header=False)
            numbers = []
            responses = []
        print(item[1]["queries"])
        
if item[0] % 5 != 0:
    temp_df = pd.DataFrame({"index": numbers, "responses": responses})
    temp_df.to_csv('responses.csv', mode='a', index=False, header=False)
    print("last set saved")


11it [00:12,  1.17s/it]


KeyboardInterrupt: 

In [2]:
# merge response to the main dataframe
response_df = pd.read_csv("responses.csv", index_col=0, header=None)
response_df.rename(columns={1: "generated_response"}, inplace=True)
df = pd.read_csv("df_checkpoint_1.csv", index_col=None)
df["generated_responses"] = response_df["generated_response"]
df.to_csv("df_checkpoint_2.csv", index=None)



NameError: name 'pd' is not defined

In [3]:
df = pd.read_csv("df_checkpoint_2.csv")
df

,queries,answers,functions,arguments,callable,generated_responses
0,What's the weather forecast for a snowy day in...,"{""function"": ""get_weather_info"", ""arguments"": ...",get_weather_info,"{""city"": ""Juneau""}","{""city"": ""Juneau"", ""time_in_PST"": ""2024-10-24 ...","The given information is for a sunny day, not ..."
1,"Is it sunny today in PST in Bangor, Maine?","{""function"": ""get_weather_info"", ""arguments"": ...",get_weather_info,"{""city"": ""Bangor""}","{""city"": ""Bangor"", ""time_in_PST"": ""2024-10-24 ...","No, it's windy in Bangor, Maine, with a temper..."
2,Current time and weather info in Kuching,"{""function"": ""get_weather_info"", ""arguments"": ...",get_weather_info,"{""city"": ""Kuching""}","{""city"": ""Kuching"", ""time_in_PST"": ""2024-10-24...",It's currently 2024-10-24 21:04:07 in Kuching....
3,Can you give me an update on the weather in In...,"{""function"": ""get_weather_info"", ""arguments"": ...",get_weather_info,"{""city"": ""Indianapolis""}","{""city"": ""Indianapolis"", ""time_in_PST"": ""2024-...",It's currently cloudy in Indianapolis with a t...
4,What is the time and temperature like right no...,"{""function"": ""get_weather_info"", ""arguments"": ...",get_weather_info,"{""city"": ""Funafuti""}","{""city"": ""Funafuti"", ""time_in_PST"": ""2024-10-2...",It's currently 2024-10-24 21:04:07 PST in Funa...
...,...,...,...,...,...,...
1675,What is the area of a rectangle with length 7....,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""7.899"", ""width"": ""8"", ""round_resul...","{""area"": ""63""}","To find the area, multiply length and width: 7..."
1676,What is the area of a square with length 2.615...,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""2.615"", ""width"": ""8"", ""round_resul...","{""area"": ""21""}","To find the area, multiply the length and widt..."
1677,Calculate the area of a square with length 8.9...,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""8.901"", ""width"": ""None"", ""round_re...","{""error"": ""Width must be a number.""}",Area = side² = 8.901² ≈ 79.3 (rounded to the n...
1678,What is the area of a rectangle with length 5....,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""5.005"", ""width"": ""8"", ""round_resul...","{""area"": ""40""}","To find the area, multiply length and width: 5..."


In [ ]:
# reverse prompt engineering
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_aws.embeddings import BedrockEmbeddings
from langchain_community.utils.math import cosine_similarity
import numpy as np
import boto3
from langchain_aws import ChatBedrock
from botocore.config import Config
import time
from tqdm import tqdm
import random
import traceback



retry_config = Config(
    retries = {
        'max_attempts': 20,  # Customize the number of retry attempts
        'mode': 'adaptive'   # Or 'adaptive' for dynamically adjusting retries
    }
)

# Initialize the Bedrock client with retry configuration
bedrock_client = boto3.client('bedrock-runtime', config=retry_config)

# Use the ChatBedrock model in LangChain with the client that has retries

llm_reverse = ChatBedrock(
    client=bedrock_client,
    model_id="meta.llama3-1-70b-instruct-v1:0",  # Or "meta.llama3-1-70b-instruct-v1:0"
    temperature=0.2,
    max_tokens=None)

class Query(BaseModel):
    """reverse prompt engineering output"""
    questions: List[str] = Field(description="a list of questions according to the user repsonse")

parser = JsonOutputParser(pydantic_object=Query)

def get_reverse_prompt(n, df, llm):
    prompt = PromptTemplate(
        template="Generate " + str(n) + " most possible questions based on the answer. Please provide a JSON response strictly according to the schema without any additional comments or variations.\n{format_instructions}\n{answer}\n",
        input_variables=["answer"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    prompt_and_model = prompt | llm
    output = prompt_and_model.invoke({"answer": df["generated_responses"]})

    return parser.invoke(output)

def calculate_mean_similiarity(query_response, reverse_queries):
    model_id = "amazon.titan-embed-text-v2:0"
    # def get_cos_sim(query_response, reverse_queries):
    embedding = BedrockEmbeddings(model_id=model_id)
    original_embedded = embedding.embed_query(query_response["query"])
    guess_embeded = embedding.embed_documents(reverse_queries["query"])
    return np.array([cosine_similarity([original_embedded], [item])[0] for item in guess_embeded]).mean()

responses = []
numbers = []
# os.remove("reverse_queries.csv")
for item in tqdm(df.iloc[921:].iterrows()):
    try:
        reverse_queries = get_reverse_prompt(3, item[1], llm_reverse)
        queries_list = json.dumps(reverse_queries["questions"])
        responses.append(queries_list)
        numbers.append(item[0])
        time.sleep(random.random())
        if item[0] % 5 == 0 :
            temp_df = pd.DataFrame({"index": numbers, "reverse_queries": responses})
            temp_df.to_csv('reverse_queries.csv', mode='a', index=False, header=False)
            numbers = []
            responses = []
            
    except Exception as e:
        print(e)
        time.sleep(random.random()*70)
        reverse_queries = get_reverse_prompt(3, item[1], llm_reverse)
        queries_list = json.dumps(reverse_queries["questions"])
        responses.append(queries_list)
        numbers.append(item[0])
        time.sleep(random.random())
        if item[0] % 5 == 0 :
            temp_df = pd.DataFrame({"index": numbers, "reverse_queries": responses})
            temp_df.to_csv('reverse_queries.csv', mode='a', index=False, header=False)
            numbers = []
            responses = []
        print(item[1]["queries"])
        
if item[0] % 5 != 0:
    temp_df = pd.DataFrame({"index": numbers, "reverse_queries": responses})
    temp_df.to_csv('reverse_queries.csv', mode='a', index=False, header=False)
    print("last set saved")




81it [09:11,  7.51s/it]ERROR:root:Error raised by bedrock service: An error occurred (ThrottlingException) when calling the InvokeModel operation: Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.


An error occurred (ThrottlingException) when calling the InvokeModel operation: Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.


82it [09:42, 14.80s/it]

What is the count of 'gallimaufry', for the letter 'm'


87it [09:55,  4.51s/it]ERROR:root:Error raised by bedrock service: An error occurred (ThrottlingException) when calling the InvokeModel operation: Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.


...

An error occurred (ThrottlingException) when calling the InvokeModel operation: Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.


113it [13:44, 15.08s/it]

What is the occurrence of the letter 'u' in the word 'universe'


120it [14:03,  4.06s/it]ERROR:root:Error raised by bedrock service: An error occurred (ThrottlingException) when calling the InvokeModel operation: Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.


An error occurred (ThrottlingException) when calling the InvokeModel operation: Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.


In [70]:
reverse_queries

{'questions': ["What is the count of the letter 'P' in 'Physikalische Chemie'?",
  "How many times does the letter 'P' appear in 'Physikalische Chemie'?",
  "What is the frequency of the letter 'P' in 'Physikalische Chemie'?"]}